POPS_binary_extract-manual.ipynb <br>
Author: Ricardo L. Pena <br>
date: 9/14/2023 <br>
Department of Chemistry <br>
Colorado State University <br>
penar@colostate.edu

# Import functions/modules

In [1]:
import numpy as np
import pandas as pd
import struct
import time
from itertools import chain
import glob
from datetime import datetime
import os
import os.path
from tqdm import tqdm

from argparse import ArgumentParser
from multiprocessing import Pool

# Functions
## Binary Extraction Functions
The functions listed below were provided by Dr. Rainwater at Handix Scientific. Permission was granted to me to use and publish the fast_flatten and extract_binary functions.
* fast_flatten: Reduces or concatanates list of lists (list = [[member1 ],[member2 ],[member3 ],...]) into a single list (list = [member1, member2, member3, ...]).
* extract_binary: extracts raw peak amplitudes and epoch timestamps from binary (.b) files and returns simple lists of the amplitudes and timestamps called PeakAmplitude and Timestamp.

In [2]:
### flatten list function
def fast_flatten(input_list):
    return list(chain.from_iterable(input_list))

### extract binary function
def extract_binary(binary_filename):
    with open(binary_filename, mode='rb') as file: # b is important -> binary
        fileContent = file.read()
    lst = [fileContent]

    file_length = len(fileContent) # full length of file
    line_index_start = 0 # initialize the index to start at zero
    list_peakamplitude, list_timestamps = [], [] # empty the lists

    while file_length>line_index_start:
        line_index_end = line_index_start+12 # do this here to not have to do it again
        num_records, timestamp_val = struct.unpack('<Id',fileContent[line_index_start:line_index_end])
        num_elements = num_records*3
        line_record_end = line_index_end+(num_elements*4)

        # get all the data out of the binary record here and into a numpy array
        # might be a faster way to do this

        chunk_data = struct.unpack('I'*num_elements,fileContent[line_index_end:line_record_end])
        chunk_data = np.asarray(chunk_data).reshape(-1,3) # turn into an array and reshaped, could be done in a single step

        # make an empty array to do the timestamp shenanigans
        # dt = time between each record except for the first entry
        # first dt is the time since the timestamp_val, float time

        chunk_timestamp = np.empty(len(chunk_data),dtype='float64') # empty array with float precision
        chunk_timestamp = np.cumsum(chunk_data[:,2]/1e6) # cumulative sum of the dt values, divide by 1e6 is because dt is in microseconds
        chunk_timestamp += timestamp_val # adding the initial timestamp of the data record to get the true timestamp

        # list comprehension is faster than numpy
        chunk_timestamp = chunk_timestamp[:].tolist() # converting to list
        chunk_amplitude = chunk_data[:,0].tolist() # converting to a list

        # the two outputs PeakAmplitude and the Timestamp
        # these are lists of lists
        list_peakamplitude.append(chunk_amplitude)
        list_timestamps.append(chunk_timestamp)
        line_index_start = line_record_end # reset things

    # at the end of this the lists of lists need flattening
    t = time.time()
    PeakAmplitude = fast_flatten(list_peakamplitude)
    Timestamp = fast_flatten(list_timestamps)
    elapsed = time.time() - t
    del fileContent; del lst
    return PeakAmplitude,Timestamp

## User input function
* input_arguments: Enables and manages user inputs using the ArgumentParser module. This only works with python (.py) scripts and not jupyter notebooks (ipynb). Argument Parser turns a python script into a user interactive command line program that can record user inputs.

    One of the user arguments is nbins which changes the number of bins we wish to bin the scatter peaks/counts into.

In [3]:
def input_arguments(): # arguments that are user defined
    # run script + -h to list current ports and see arguments.
    parser = ArgumentParser()
    parser.add_argument('--directory', type = str,
        help = 'Select the directory containing binary files or the directory containing subdirectories containing binaries.',
        default=".")
    parser.add_argument('--nbins', type = int,
        help = 'Select the desired number of bins (default = 16 bins).',
        default=16)
    parser.add_argument('--mie', type = str,
        help = 'Select the smoothed mie conversion table to interpolate bins from (default = Mie_scripps_1_41.csv).',
        default='/Users/FarmerLab/Documents/python_scripts/mie_conversion_tables/Mie_scripps_1_41.csv')
    parser.add_argument('--multiproc', action='store_true', help= 'call argument to enable binary processing with four processors.')
    parser.add_argument('--logmin', type = float,
        help = 'log of the amplitude corresponding to 120 nm in mie table (default = 0.871 for Scripps)',
        default=0.871)
    parser.add_argument('--logmax', type = float,
        help = 'log of the amplitude corresponding to 3000 nm in mie table (default = 4.067 for Scripps)',
        default=4.067)

    arguments = parser.parse_args()
    return arguments

## Mie Bin-Creation Functions
* bin_headers: calculate the log amplitude bin edges based on a an equation provided in the POPS technical manual. The specific equation is log_bin_edge = logmin + ((logmax-logmin)/nbins)*bin_number.

In [4]:
# Bin set-up and convertion based on Mie conversion table
def bin_headers(nbins, logmin, logmax):
    #replicating mie conversion excel sheet sent by Dr. Bryan Rainwater at Handix
    # The number of bins, logmin, and log max can change.
    upper_bin_edge = []
    lower_bin_edge = []
    for i in range(0,nbins):
        upper_n = i + 1
        lower_n = i

        l = logmin + ((logmax-logmin)/nbins)*lower_n
        u = logmin + ((logmax-logmin)/nbins)*upper_n
        # bin edges are reported as loggeed values; 10**bin_edge to get non-logged intensity/bin edges
        lower_bin_edge.append(l)
        upper_bin_edge.append(u)
    return lower_bin_edge, upper_bin_edge

* linear_interpolation: use the linear slope between two points to find the y value of a given x value between the two points.

In [5]:
def linear_interpolation(x_lower,y_lower,x_upper,y_upper,x_interp, logged):
    if logged == True: np.log10(x_interp)
    y_interp = y_lower + ((x_interp - x_lower)*((y_upper-y_lower)/(x_upper-x_lower)))
    return y_interp

* intensity_2_diameter: convert bin edges listed as raw amplitudes to nm particle diameters given a mie conversion table to account for varying index of refraction.

    for each amplitude bin edge:
    1. Check if the provided mie conversion table lists logged amplitudes or raw amplitudes. We assume that Log_Intensity is false which means that our mie conversion table lists raw peak amplitudes/intensities. As a result, we take the log10 of our mie table amplitudes such that we can directly compare to our logged bin edges yielded by the bin_headers function.
     - if the mie table already displays logged amplitudes, then we use the table as is.
    2. Scan through each of the mie table amplitudes (IOR_raw_amp) and see which amplitudes encompass our bin edge. We start scanning mie amplitudes from smallest to largest such that for a bin edge located somewhere in the middle of our amplitude range, the bin edge amplitude is greater than our mie amplitudes. Thus, the initial bottom and top bools are True and False. When we get to mie amplitudes greater than our bin edge, the bools switch to become False and True for bottom and top. The mie table amplitude for-loop breaks (stops) at the first instance where the bools switch. The break index is recorded as the index of the upper mie amplitude and the index-1 is recorded for the lower mie amplitude. 
    3. These indices are then used to identify the mie table amplitudes that surround the bin edge along with the upper and lower diameters that correspond to the upper and lower mie amplitudes we identified.
    4. The bin edge is converted from a logged value back to a normal by setting the bin edge as the exponent of 10.
    5. Then, using the upper and lower amplitudes and diameters, we can linearly interpolate a diamter for our amplitude bin edge.
    6. The bin edge and its interpolated diameter are recorded to the raw amplitudes and interp_diameter lists.
    7. the above steps are repeated for the next bin edge in our list of bin edges from the bin_headers function.

<center><img src="IMG_2433.jpg" alt="Intensity_2_diameter" width="25%" height="auto" class="blog-image"/></center>

In [6]:
def itensity_2_diameter(bin_range, Mie_map_amp, Mie_map_diam, Log_Intensity):
    raw_amplitude, interp_diameter = [], []
    for signal_amp in bin_range:
        try:
            IOR_raw_amp = np.log10(Mie_map_amp) # immediately take log to compare to logged bin edges(signal_amp)
            # bool = is amp value in mie table the log intensity of the amplitude or the raw amplitude?
            # True indicates that the mie table displays logged amplitudes or intensities.
            # False indicates that raw amplitudes/intensities are displayed in conversion table.
            if Log_Intensity == True: IOR_raw_amp = Mie_map_amp

            for i, amp in enumerate(IOR_raw_amp):
                    bottom = bool(signal_amp >= amp)
                    top = bool(signal_amp <=amp)

                    if ((bottom == False) and (top == True)):
                        upper_idx = i
                        lower_idx = i-1
                        break
            upper_amp, lower_amp =  Mie_map_amp[upper_idx], Mie_map_amp[lower_idx]
            upper_diam, lower_diam = Mie_map_diam[upper_idx], Mie_map_diam[lower_idx]

            # x is amp and y is diameter
            #if Log_Intensity == False: signal_amp = 10**signal
            signal = 10**signal_amp #the signal_amp is always logged, but log needs to be undone to correctly bin raw intensities which are not logged.
            signal_diam = linear_interpolation(lower_amp, lower_diam, upper_amp, upper_diam, signal, Log_Intensity)
            raw_amplitude.append(signal)
            interp_diameter.append(signal_diam)
            #print(signal, signal_diam)

        except Exception as e:
            print("outside of Mie amp range")
            print(e)
    return raw_amplitude, interp_diameter

* create_bins: Returns a mie table containing bin edge amplitudes, bin edge diameter, the mean bin diameter, and header labels.

    mie table:
    1. Calculated bin edges (intensity) and their corresponding diameters are curated into a table. The lower bin edges are added to a column called "Lower Amp" while the upper bin edges are added to a column called "Upper Amp". The interpolated bin edges for the upper and lower amplitudes are labeled "Lower Bin Diameter" and "Upper Bin Diameter" in the table. 
    2. The center (mean) bin diameter is calculated and added to a column in the dataframe called "Mean Diameter (nm)". 
    3. The diameter bin edges are concatenated togther in a string to be used a table headers. The strings are placed in the "Bin Header" column.

In [7]:
# Calculate log intensity bin edges for desired bin number
def create_bins(nbins,logmin,logmax, mie_conv_table_intensity, mie_conv_table_diam, logged_amplitudes):
    l_bin, u_bin = bin_headers(nbins, logmin, logmax)

    upper_raw_amp, upper_interp_diam = itensity_2_diameter(u_bin,mie_conv_table_intensity, mie_conv_table_diam, logged_amplitudes)
    lower_raw_amp, lower_interp_diam = itensity_2_diameter(l_bin, mie_conv_table_intensity, mie_conv_table_diam,logged_amplitudes)
    mie_smooth_dict = {"Lower Amp": lower_raw_amp, "Upper Amp": upper_raw_amp,
                "Lower Bin Diameter": lower_interp_diam, "Upper Bin Diameter": upper_interp_diam}
    mie_smooth_df = pd.DataFrame(mie_smooth_dict)
    mie_smooth_df['Mean Diameter (nm)'] = mie_smooth_df[['Lower Bin Diameter','Upper Bin Diameter']].mean(axis=1)
    mie_smooth_df['Bin Header'] = round(mie_smooth_df['Lower Bin Diameter'],2).astype('str') + "_" + round(mie_smooth_df['Upper Bin Diameter'],2).astype('str')
    return mie_smooth_df

# Main Code
## Main working functions
* binary_2_csv: Produces a 10 Hz, 30-minute binned counts file from a singular binary file containing raw peak amplitudes and epoch times.

In [8]:
## Main working functions
def binary_2_csv(binary_mie_bins_info): # main working function
    # retrieve binary file to process and the bins and mie table to use
    binary_file = binary_mie_bins_info[0]
    mie_table = binary_mie_bins_info[1]
    bins = binary_mie_bins_info[2]

    file_csv_name = binary_file.split('.b')[0]+'_10Hz.csv'
    ### check if file already exists before extracting the binary again ###
    if (os.path.isfile(file_csv_name) == False):
        print('created {}'.format(file_csv_name))
        peaks, time_s = extract_binary(binary_file)

        # (1) create a raw dataframe from binary data
        raw_peak_df = pd.DataFrame({'timestamp': time_s,'Peaks':peaks}) #.to_csv(file_csv_name)
        #raw_peak_df.to_csv(binary_file.split('.b')[0]+'_raw.csv') ### binning check!
        raw_peak_df['timestamp'] = raw_peak_df['timestamp'].apply(lambda x: datetime.utcfromtimestamp(x)) #recently updated
        timing_df = raw_peak_df.set_index('timestamp') # set timestamp as index

        # (2) resampling the data to 10Hz
        try:
            df_10Hz = timing_df.groupby(pd.Grouper(freq='100ms'))['Peaks'].apply(list)
            #df_10Hz.to_csv(binary_file.split('.b')[0]+'_grouped-10Hz.csv') ### binning check!
            # bin and count the data now
            bin_counts = df_10Hz.apply(lambda x: pd.cut(x, bins=bins).value_counts())
            bin_counts.columns = mie_table['Bin Header']
            bin_counts.to_csv(file_csv_name)

            ### added in these lines to visualize df's created in steps 1 and 2
            return raw_peak_df, df_10Hz, bin_counts

            lst_p = [peaks, time_s, raw_peak_df, timing_df, df_10Hz, bin_counts]
            del peaks; del time_s; del raw_peak_df; del timing_df; del df_10Hz; del bin_counts
            del lst_p
        except: print("Fatal error in file: ",file_csv_name)

    else:
        print ('{} already exists!'.format(file_csv_name))
        pass

    lst = [binary_file, mie_table, bins]
    del binary_file; del mie_table; del bins
    del lst

* main: Gathes the user input information to develope the desired number of bins and bin labels from a provided mie table. The bins are used then for processing with binary_2_csv provided that the 10Hz file doesn;t exist already. Multiple binary files can be processed at the same time with multiproc or individualy.

In [9]:
def main(args):
    ### User defined arguments
    num_bins = args.nbins #16 # turn into arg
    logmin = args.logmin
    logmax = args.logmax
    mie_input_csv = args.mie
    working_dir = args.directory
    run_multiprocessing = args.multiproc

    ### define Mie Table based on num bins:
    # logmin and logmax change depending on the mie table used.
    # look at the amplitude for 120 nm and 3000 nm and take log10 of amplitude
    
    #"Scripps:
    # logmin = 0.871; 120 nm
    # logmax = 4.067; 3000 nm"
 
    # "chestnutridge
    # logmin = 1.526; 120 nm
    # logmax = 4.562; 3000nm"


    # also turn mie conversion table into arg
    mie_conv_table = pd.read_csv(mie_input_csv)
    table_amp = mie_conv_table['amp_scale']
    table_diam = mie_conv_table['d_nm']
    mie_table = create_bins(num_bins,logmin,logmax,table_amp, table_diam, False)
    mie_table.to_csv(working_dir+"/resulting_mie-table.csv") ### binning check!
    bins = (list(mie_table['Lower Amp'])
    + list(mie_table['Upper Amp'][-1:]))

    ### Walk subdirectories for all binary files ###
    source_directory = working_dir
    # create list of binary files with complete path included
    binary_files_2_process = []
    for dirpath, dirnames, filenames in os.walk(source_directory):
        for filename in [f for f in filenames if f.endswith(".b")]:
            full_path = os.path.join(dirpath, filename)
            binary_files_2_process.append(full_path)
    binary_files_2_process.sort(key=os.path.getmtime)

    print('Extracting POPS binaries...')
    binary_mie_bins = [(f, mie_table, bins) for f in binary_files_2_process] # single arg for pool

    ### Perform extraction is pools of 4 (four processes at once) ###
    if run_multiprocessing == True:
        print('Multiprocessing enabled... 4 procs being used.')
        p = Pool(4)
        p.map(binary_2_csv,binary_mie_bins)
    else:
        for group in binary_mie_bins:
            try:
                binary_2_csv(group) # Main function that produces binned count csv's
            except Exception as e:
                print("File Error!")
                print("{} failed".format(group))
                print("error:\n", e)
    print('done.')

## Make Python Executable
execute the main functions

In [10]:
### Execute everything as a python script
# if __name__ == '__main__':
#     args = input_arguments()
#     main(args)

# Running the code
Delete the Peak_*_10Hz.csv's prior to running this code.

In [13]:
# Hard coded arguments
# num_bins = args.nbins #16 # turn into arg
# logmin = args.logmin
# logmax = args.logmax
# mie_input_csv = args.mie
# working_dir = args.directory
# run_multiprocessing = args.multiproc

num_bins = 16
logmin = 0.871
logmax = 4.067
mie_input_csv = '../mie_conversion_tables/Mie_scripps_1_41.csv'
working_dir = './Example_data/POPS_F20230707/'

# keep false when running in jupyter notebook. Some additions have been made to 
# this code to visualize data in jupyter lab/notebook. Use the original code if
# you desire to run the python executable.
run_multiprocessing = False

In [14]:
# Produce mie table dataframe
mie_conv_table = pd.read_csv(mie_input_csv)
table_amp = mie_conv_table['amp_scale']
table_diam = mie_conv_table['d_nm']
mie_table = create_bins(num_bins,logmin,logmax,table_amp, table_diam, False)
mie_table.to_csv(working_dir+"/resulting_mie-table.csv") ### binning check!
bins = (list(mie_table['Lower Amp'])
+ list(mie_table['Upper Amp'][-1:]))

# Display calculated information
print('list of bin edges reported as raw intensities:')
print(*bins, sep = "\n")
print('\nTable of bin edges:')
mie_table

list of bin edges reported as raw intensities:
7.430191378967014
11.769282841042024
18.64232178252499
29.52908568322213
46.77351412871981
74.08836316231077
117.35456824912899
185.88742011708283
294.4421633798763
466.3908268844408
738.7542629936365
1170.1728027907702
1853.5316234148124
2935.9590915163635
4650.503761666063
7366.310143681267
11668.09617060963

Table of bin edges:


,Lower Amp,Upper Amp,Lower Bin Diameter,Upper Bin Diameter,Mean Diameter (nm),Bin Header
0,7.430191,11.769283,120.025342,130.818086,125.421714,120.03_130.82
1,11.769283,18.642322,130.818086,143.198798,137.008442,130.82_143.2
2,18.642322,29.529086,143.198798,157.782008,150.490403,143.2_157.78
3,29.529086,46.773514,157.782008,175.575915,166.678962,157.78_175.58
4,46.773514,74.088363,175.575915,198.075846,186.825881,175.58_198.08
5,74.088363,117.354568,198.075846,227.168410,212.622128,198.08_227.17
6,117.354568,185.887420,227.168410,270.145591,248.657000,227.17_270.15
7,185.887420,294.442163,270.145591,351.087673,310.616632,270.15_351.09
8,294.442163,466.390827,351.087673,479.997437,415.542555,351.09_480.0
9,466.390827,738.754263,479.997437,576.977273,528.487355,480.0_576.98


In [15]:
 ### Walk subdirectories for all binary files ###
source_directory = working_dir
# create list of binary files with complete path included
binary_files_2_process = []
for dirpath, dirnames, filenames in os.walk(source_directory):
    for filename in [f for f in filenames if f.endswith(".b")]:
        full_path = os.path.join(dirpath, filename)
        binary_files_2_process.append(full_path)
binary_files_2_process.sort(key=os.path.getmtime)

# Display calculated information
print('Ordered list of binary files:')
print(*binary_files_2_process, sep = "\n")

Ordered list of binary files:
./Example_data/POPS_F20230707/Peak_20230707x001.b
./Example_data/POPS_F20230707/Peak_20230707x002.b
./Example_data/POPS_F20230707/Peak_20230707x003.b
./Example_data/POPS_F20230707/Peak_20230707x004.b
./Example_data/POPS_F20230707/Peak_20230707x005.b
./Example_data/POPS_F20230707/Peak_20230707x006.b
./Example_data/POPS_F20230707/Peak_20230707x007.b
./Example_data/POPS_F20230707/Peak_20230707x008.b
./Example_data/POPS_F20230707/Peak_20230707x009.b


In [16]:
# Run on multiple processors (idk if this will work on jupyter)
binary_mie_bins = [(f, mie_table, bins) for f in binary_files_2_process] # a single argument is needed for pool

### Perform extraction is pools of 4 (four processes at once) ###
# do not run the multiprocessing section, Keep run_multiprocessing = False
if run_multiprocessing == True:
    print('Multiprocessing enabled... 4 procs being used.')
    p = Pool(4)
    p.map(binary_2_csv,binary_mie_bins)

### run the binary extraction and binning code on each binary file
else:
    for group in tqdm(binary_mie_bins):
        try:
            raw_peak_df, df_10Hz, bin_counts = binary_2_csv(group)
        except Exception as e:
            print("File Error!")
            print("{} failed".format(group))
            print("error:\n", e)
print('done.')

  0%|          | 0/9 [00:00<?, ?it/s]

created ./Example_data/POPS_F20230707/Peak_20230707x001_10Hz.csv


 11%|█         | 1/9 [00:09<01:18,  9.76s/it]

created ./Example_data/POPS_F20230707/Peak_20230707x002_10Hz.csv


 22%|██▏       | 2/9 [00:09<00:28,  4.14s/it]

...

created ./Example_data/POPS_F20230707/Peak_20230707x008_10Hz.csv


 89%|████████▉ | 8/9 [01:59<00:30, 30.29s/it]

created ./Example_data/POPS_F20230707/Peak_20230707x009_10Hz.csv


100%|██████████| 9/9 [02:22<00:00, 15.82s/it]

done.


### A look into the bin creation steps

In [17]:
# STEP1: The raw data frame generated from decoding the binary files
raw_peak_df

,timestamp,Peaks
0,2023-07-07 21:36:09.998292,71
1,2023-07-07 21:36:09.998303,94
2,2023-07-07 21:36:09.998313,83
3,2023-07-07 21:36:09.998325,79
4,2023-07-07 21:36:09.998329,83
...,...,...
287092,2023-07-07 21:55:19.215172,80
287093,2023-07-07 21:55:19.216175,106
287094,2023-07-07 21:55:19.216960,177
287095,2023-07-07 21:55:19.229364,188


In [18]:
# STEP2:10 Hz grouped peak data
df_10Hz

timestamp
2023-07-07 21:36:09.900     [71, 94, 83, 79, 83, 100, 76, 109, 78, 110, 342]
2023-07-07 21:36:10.000    [135, 175, 303, 77, 655, 569, 270, 126, 266, 1...
2023-07-07 21:36:10.100    [1037, 103, 180, 177, 113, 1273, 335, 116, 663...
2023-07-07 21:36:10.200    [186, 641, 115, 129, 88, 941, 728, 94, 210, 10...
2023-07-07 21:36:10.300    [679, 195, 271, 861, 108, 106, 217, 239, 159, ...
                                                 ...                        
2023-07-07 21:55:18.800    [244, 263, 187, 190, 310, 799, 133, 500, 448, ...
2023-07-07 21:55:18.900    [116, 818, 105, 228, 287, 155, 490, 1136, 283,...
2023-07-07 21:55:19.000    [454, 294, 127, 749, 814, 796, 108, 282, 566, ...
2023-07-07 21:55:19.100    [114, 182, 231, 167, 737, 341, 332, 154, 476, ...
2023-07-07 21:55:19.200              [731, 140, 325, 80, 106, 177, 188, 102]
Freq: 100L, Name: Peaks, Length: 11494, dtype: object

In [19]:
#STEP3: binned data
bin_counts

Bin Header,120.03_130.82,130.82_143.2,143.2_157.78,157.78_175.58,175.58_198.08,198.08_227.17,227.17_270.15,270.15_351.09,351.09_480.0,480.0_576.98,576.98_687.6,687.6_905.18,905.18_1193.71,1193.71_1720.62,1720.62_2324.8,2324.8_3000.03
timestamp,,,,,,,,,,,,,,,,
2023-07-07 21:36:09.900,0,0,0,0,1,9,0,0,1,0,0,0,0,0,0,0
2023-07-07 21:36:10.000,0,0,0,0,0,5,5,4,4,3,0,0,0,0,0,0
2023-07-07 21:36:10.100,0,0,0,0,0,5,5,8,5,3,4,1,0,0,0,0
2023-07-07 21:36:10.200,0,0,0,0,0,8,7,8,6,8,4,0,0,0,0,0
2023-07-07 21:36:10.300,0,0,0,0,0,5,6,10,5,3,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-07 21:55:18.800,0,0,0,0,1,6,5,5,6,6,7,0,0,0,0,0
2023-07-07 21:55:18.900,0,0,0,0,0,4,2,5,3,3,3,0,1,0,0,0
2023-07-07 21:55:19.000,0,0,0,0,1,7,4,10,5,4,5,0,0,0,0,0
